In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,BatchNormalization,Dropout,Flatten,Input
from tensorflow.keras import optimizers
from keras import Model
from tensorflow.keras import initializers
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.backend import clear_session

In [19]:
df = pd.read_csv('../input/dataset/train.csv')
df

In [ ]:
for i in df.columns[1:]:
    print(df[i].value_counts())

In [20]:
le_age = LabelEncoder()
le_race = LabelEncoder()
ages = le_age.fit_transform(df['age'])
races= le_race.fit_transform(df['race'])
df=pd.get_dummies(df, columns=['gender'], drop_first=True)

pair_age={idx:age for idx, age in enumerate(le_age.classes_)}
pair_race={idx:race for idx, race in enumerate(le_race.classes_)}
df['race_enc'] = races
df['age_enc'] = ages

df=df.sample(frac=1).reset_index()
pair_age, pair_race

In [21]:
y=df.iloc[:5000,-3:]
x=df['file'][:5000]

from sklearn.model_selection import train_test_split
x1,x2,y1,y2=train_test_split(x, y, test_size=0.35, random_state=47, shuffle=True)
# Here, 1 being the train data and 2 being the test data

x1.shape, x2.shape, y1.shape, y2.shape

In [ ]:
for i in y1.columns:
    print(y1[i].value_counts())

In [22]:
datagen = ImageDataGenerator(rotation_range=35, zoom_range=0.2, horizontal_flip=True,
                            brightness_range =(0.2,0.5),height_shift_range=0.2,
                             width_shift_range=0.2,rescale=1/255.)

folder_path = '../input/dataset/'
label_df = y1.reset_index(drop=True)
from tqdm import tqdm
images = np.empty((0,80,80,3))
labels = pd.DataFrame()
for i in tqdm(range(len(x1))):
    a=load_img(folder_path+x1.reset_index().file[i], target_size=(80,80))
    a=img_to_array(a)
    val = label_df['age_enc'][i] 
    flow = datagen.flow(a.reshape((1,80,80,3)), batch_size=1)
    if val== 1 or val==6:
        for j in range(2):
            aug = flow.next()
            images = np.append(images, aug, axis=0)
            labels = pd.concat([labels, label_df.iloc[i:i+1,:]])
            
    if val==7:
        for j in range(6):
            aug = flow.next()
            images = np.append(images, aug, axis=0)
            labels = pd.concat([labels, label_df.iloc[i:i+1,:]])
                       
    if  val==0 or val==8:
        for j in range(10):
            aug = flow.next()
            images = np.append(images, aug, axis=0)
            labels = pd.concat([labels, label_df.iloc[i:i+1,:]])
    else:
        a=a/255.
        images = np.append(images, a.reshape([1,80,80,3]), axis=0)
        labels = pd.concat([labels, label_df.iloc[i:i+1,:]])


In [23]:
from sklearn.utils import shuffle
images,labels = shuffle(images, np.array(labels), random_state=59)

In [67]:
ran = np.random.randint(0,len(images),1)
print(ran)
vals =labels[ran].flatten()

plt.imshow(images[ran][0])
if vals[0] ==1:
    g='Male'
else:g='Female'
plt.title(g + pair_race[vals[1]] + pair_age[vals[2]])

In [24]:
test_images=[]
for file in tqdm(x2):
    img = load_img(folder_path + file, target_size=(80,80))
    img = img_to_array(img)/255.
    test_images.append(img)

test_images = np.array(test_images)

In [25]:
from tensorflow.keras.applications import InceptionResNetV2
conv_base4 = InceptionResNetV2(include_top=False,weights="imagenet",
                               input_shape=(80,80,3))

In [26]:
model1 = Sequential()
model1.add(conv_base4)
conv_base4.trainable=False
model1.summary()

In [27]:
trx=[]
for i in tqdm(range(len(images))):
    a=model1.predict(images[i].reshape([1,80,80,3])).flatten()
    trx.append(a)
trx=np.array(trx)

In [28]:
tex=np.empty([0,1536])
for i in tqdm(range(len(test_images))):
    b=model1.predict(test_images[i].reshape([1,80,80,3])).reshape([1,1536])
    tex=np.append(tex,b,axis=0)

In [29]:
from catboost import CatBoostClassifier
from sklearn.multioutput import MultiOutputClassifier

In [31]:
clf = MultiOutputClassifier(CatBoostClassifier(iterations=1000))
clf.fit(trx, labels)

In [60]:
pred=[]
for i in tex:
    p = clf.predict(i)
    pred.append(p)

In [61]:
r1=[]
for i in tqdm(range(len(tex))):
    if pred[i][0] == np.array(y2)[:,0][i]:
        r1.append(1)
    else: r1.append(0)
        
r2=[]
for i in tqdm(range(len(tex))):
    if pred[i][1] == np.array(y2)[:,1][i]:
        r2.append(1)
    else: r2.append(0)
        
r3=[]
for i in tqdm(range(len(tex))):
    if a[i][2] == np.array(y2)[:,2][i]:
        r3.append(1)
    else: r3.append(0)
        
r3.count(1)/len(r1)

In [65]:
print('Test Accuracy of gender :' ,r1.count(1)/len(r1))
print('Test Accuracy race:' ,r2.count(1)/len(r1))
print('Test Accuracy age:' ,r3.count(1)/len(r1))

In [38]:
import pickle
file = open('fairfc.pkl', 'wb')
pickle.dump(clf,file)